In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import the required libraries

In [ ]:
import pandas as pd
import numpy as np
import keras
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Read the Data

In [ ]:
df_train=pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Train.csv')
df_val=pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Valid.csv')
df_train.head()

In [ ]:
df_val.head()

In [ ]:
X_train=df_train['text'].values
Y_train=df_train['label'].values

In [ ]:
X_val=df_val['text'].values
Y_val=df_val['label'].values

In [ ]:
(X_train.shape,Y_train.shape),(X_val.shape,Y_val.shape)

# Analyse the Data

In [ ]:
df_train.iloc[:,1].describe()

In [ ]:
df_val.iloc[:,1].describe()

In [ ]:
df1=pd.DataFrame(X_val_len,columns=['len'])
df1.describe()

In [ ]:
df=pd.DataFrame(X_train_len,columns=['len'])
df.describe()

In [ ]:
X_train_len=[len(str(i).split()) for i in X_train]
plt.hist(X_train_len)

In [ ]:
X_val_len=[len(str(i).split()) for i in X_val]
plt.hist(X_val_len)

# Setting the parameters

In [ ]:
vocab_size=30000 #went for an average vocab size 
embedding_dimension=64 #high dimensions would result in finding better parameters for similarity 
max_length=120 #used a maximum length of 120 words
turnc='post'#preprocessing step for pad_sequences
oov_tok='<OOV>'#oov stands for out of vocabulary tokens

#  Tokenizing  and converting the data into Sequences

In [ ]:
tokenizer=Tokenizer(num_words=vocab_size,filters='''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~¡¢£¤¦§¨«­®°³´·º»½¾¿ßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþğıōżאגויכלמןר–‘’“”…″₤★、''',oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
X=tokenizer.texts_to_sequences(X_train)
X_padded=pad_sequences(X,maxlen=max_length,padding='post',truncating=turnc)
X_val_seq=tokenizer.texts_to_sequences(X_val)
X_val_padded=pad_sequences(X_val_seq,maxlen=max_length,padding='post',truncating=turnc)

In [ ]:
X_padded.shape,X_val_padded.shape

# The Model

In [ ]:
from keras.layers import LSTM,Bidirectional,Embedding,Dense,SpatialDropout1D,Flatten,Dropout
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_dimension,input_length=max_length))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(120,activation='tanh',return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(120,activation='tanh',return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

In [ ]:
model.compile(optimizer="rmsprop",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
hist=model.fit(X_padded,Y_train,epochs=7,batch_size=16,validation_data=(X_val_padded,Y_val))#10 #best at 9

In [ ]:
hist=model.fit(X_padded,Y_train,epochs=2,batch_size=16,validation_data=(X_val_padded,Y_val))#10 #best at 9

# This Plot is for the last two Epochs

In [ ]:
plt.plot(hist.history['accuracy'],c='b')
plt.plot(hist.history['val_accuracy'],c='r')
plt.show()

In [ ]:
plt.plot(hist.history['loss'],c='b')
plt.plot(hist.history['val_loss'],c='r')
plt.show()

# Reading the Test Data

In [ ]:
df_test=pd.read_csv('../input/imdb-dataset-sentiment-analysis-in-csv-format/Test.csv')
df_test

In [ ]:
X_test=df_test['text'].values
Y_test=df_test['label'].values

# Converting into Sequential Data

In [ ]:
X_test_seq=tokenizer.texts_to_sequences(X_test)

In [ ]:
X_test_padded=pad_sequences(X_test_seq,maxlen=max_length,padding='post',truncating=turnc)
X_test_padded[0]

In [ ]:
X_test_padded.shape

In [ ]:
model.evaluate(X_test_padded,Y_test)

# Check for your own Reviews

In [ ]:
def Check(x):
    test_case1=[x]
    test_case=tokenizer.texts_to_sequences(test_case1)
    test_case_padded=pad_sequences(test_case,padding='post',truncating=turnc)
    x=model.predict_classes(test_case_padded)
    if x==1:print("Positive", (model.predict(test_case_padded)))
    else:print("Negative", (model.predict(test_case_padded)))

In [ ]:
test_review=str(input("Enter the review :  "))
Check(test_review)

# I just checked for one random imdb review 

In [ ]:
test_review="You will get A to Z all details of this scam, i may be wrong but due to this series, i don't think that i need to watch the upcoming movie big bull because 2 to 3 hours too less for explain the story. If you are at home, just watch this web series, you will not bore for even 1 second, All actors played their role excellently, i like reporter character as w"
Check(test_review)